## Comprovació absència NA i Outliers df

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

sc = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/10 10:14:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
paths = ['data_lake/compravenda_sup.parquet', 'data_lake/rent_price.parquet', 'data_lake/renda.parquet']
col_numeriques = {'data_lake/compravenda_sup.parquet': ['Nombre']
                  , 'data_lake/renda.parquet': ['Import_Euros']
                , 'data_lake/rent_price.parquet': ['Price']}

In [15]:
for path in paths:
    df = sc.read.parquet(path)
    
    # Verificar si hay valores NA
    na_count = df.na.drop().count()
    if na_count == df.count():
        print(f"No hay valores NA en el DataFrame {path}")
    else:
        print(f"Hay valores NA en el DataFrame {path}")

No hay valores NA en el DataFrame data_lake/compravenda_sup.parquet
No hay valores NA en el DataFrame data_lake/rent_price.parquet
No hay valores NA en el DataFrame data_lake/renda.parquet


In [17]:
from pyspark.sql.types import IntegerType

In [22]:
col_numeriques

{'data_lake/compravenda_sup.parquet': ['Nombre'],
 'data_lake/renda.parquet': ['Import_Euros'],
 'data_lake/rent_price.parquet': ['Price']}

In [ ]:
for path in paths:
    dfs = sc.read.parquet(path)\
        .reduce(lambda x, y: x.union(y))
    for column, data_type in dfs.dtypes:
        if data_type != 'string':
            numeric_columns = dfs.select(column).filter(col(column).cast(IntegerType()).isNull()).count()\
                .describe(numeric_columns)

# Calcular límites para detectar outliers
outliers = {}
for column in numeric_columns:
    quantiles = merged_df.approxQuantile(column, [0.25, 0.75], 0.05)
    IQR = quantiles[1] - quantiles[0]
    lower_bound = quantiles[0] - 1.5 * IQR
    upper_bound = quantiles[1] + 1.5 * IQR
    outliers[column] = (lower_bound, upper_bound)

# Filtrar outliers
filtered_df = merged_df
for column, (lower_bound, upper_bound) in outliers.items():
    filtered_df = filtered_df.filter((col(column) >= lower_bound) & (col(column) <= upper_bound))

# Mostrar los outliers detectados
filtered_df.show()

In [51]:
numeric_columns = []
for column in col_numeriques:
    df = sc.read.parquet(column)
    for col_name in col_numeriques[column]:
        df = df.withColumn(col_name, col(col_name).cast(IntegerType()))
        
        if df.select(col_name).dtypes[0][1] == 'int':
                numeric_columns.append(col_name)
        else:
            pass